In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot
from Bio import SearchIO

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [3]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [7]:
f = proc_cfg(config['lr']['pfam']['hmmer_out'], od)

In [22]:
infile = open(f, 'r')
outfile = open('testH-hmm.tsv', 'w')

# write header to file first
header = ['transcript_id', 'accession', 'bias', 'bitscore', 'description',
          'evalue', 'id']
outfile.write('\t'.join(header)+'\n')
for record in SearchIO.parse(infile, 'hmmscan3-domtab'):
    
    tid = record.id

    for hit in record.hits:
        new_line = []
        new_line.append(tid)
        new_line.append(str(hit.accession))
        new_line.append(str(hit.bias))
        new_line.append(str(hit.bitscore))
        new_line.append(str(hit.description))
        new_line.append(str(hit.evalue))
        new_line.append(str(hit.id))
        outfile.write('\t'.join(new_line)+'\n')
infile.close()
outfile.close()

In [23]:
df = pd.read_csv('testH-hmm.tsv', sep='\t')
df.head()

,transcript_id,accession,bias,bitscore,description,evalue,id
0,transcript_100122,PF16543.10,1.7,84.8,"DRG Family Regulatory Proteins, Tma46",5.400000e-24,DFRP_C
1,transcript_100122,PF00642.29,1.3,30.1,Zinc finger C-x8-C-x5-C-x3-H type (and similar),3.900000e-07,zf-CCCH
2,transcript_100122,PF18345.6,6.1,27.3,Zinc finger domain,3.200000e-06,zf_CCCH_4
3,transcript_100122,PF16131.10,8.6,27.8,Torus domain,3.500000e-06,Torus
4,transcript_100122,PF18044.6,4.8,26.2,CCCH-type zinc finger,5.800000e-06,zf-CCCH_4


In [ ]:
# filters that ruben was using
domain_score = 10
domain_evalue = 1e-03
accuracy = 0.8
partiality = 0.9